In [1]:
import torch
import gpytorch
import botorch
import matplotlib.pyplot as plt
import numpy as np

plt.style.use("bmh")

from tqdm.notebook import tqdm

class Hartmann:
    """Generic Hartmann in 3,4,6 dims on [0,1]^d."""
    _params = {
        3: {
            "alpha": torch.tensor([1.0,1.2,3.0,3.2]),
            "A":     torch.tensor([[3.0,10,30],
                                   [0.1,10,35],
                                   [3.0,10,30],
                                   [0.1,10,35]]),
            "P":     torch.tensor([[3689,1170,2673],
                                   [4699,4387,7470],
                                   [1091,8732,5547],
                                   [381,5743,8828]]) * 1e-4
        },
        4: {
            "alpha": torch.tensor([1.0,1.2,3.0,3.2]),
            "A":     torch.tensor([[10,3,17,3.5],
                                   [0.05,10,17,0.1],
                                   [3,3.5,1.7,10],
                                   [17,8,0.05,10]]),
            "P":     torch.tensor([[1312,1696,5569,124],
                                   [2329,4135,8307,3736],
                                   [2348,1451,3522,2883],
                                   [4047,8828,8732,5743]]) * 1e-4
        },
        6: {
            "alpha": torch.tensor([1.0,1.2,3.0,3.2]),
            "A":     torch.tensor([[10,3,17,3.5,1.7,8],
                                   [0.05,10,17,0.1,8,14],
                                   [3,3.5,1.7,10,17,8],
                                   [17,8,0.05,10,0.1,14]]),
            "P":     torch.tensor([[1312,1696,5569,124,8283,5886],
                                   [2329,4135,8307,3736,1004,9991],
                                   [2348,1451,3522,2883,3047,6650],
                                   [4047,8828,8732,5743,1091,381]]) * 1e-4
        }
    }

    def __init__(self, dim=3):
        assert dim in (3,4,6)
        p = self._params[dim]
        self.alpha, self.A, self.P = p["alpha"], p["A"], p["P"]
        self.bounds = torch.stack([torch.zeros(dim), torch.ones(dim)])

    def __call__(self, x: torch.Tensor) -> torch.Tensor:
        # x: (..., d)
        d = x.size(-1)
        inner = ((x.unsqueeze(-2) - self.P)**2 * self.A).sum(dim=-1)  # (..., 4)
        return -(self.alpha * torch.exp(-inner)).sum(dim=-1)

def visualize_gp_belief_and_policy(model, likelihood, policy=None, next_x=None):
    # Disabled for 4D Hartmann function; kept for compatibility
    pass

class GPModel(gpytorch.models.ExactGP, botorch.models.gpytorch.GPyTorchModel):
    _num_outputs = 1

    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

def fit_gp_model(train_x, train_y, num_train_iters=500):
    # Declare the GP
    noise = 1e-4

    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = GPModel(train_x, train_y, likelihood)
    model.likelihood.noise = noise

    # Train the hyperparameters
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    model.train()
    likelihood.train()

    for i in tqdm(range(num_train_iters), desc="Training GP"):
        optimizer.zero_grad()

        output = model(train_x)
        loss = -mll(output, train_y)

        loss.backward()
        optimizer.step()

    model.eval()
    likelihood.eval()

    return model, likelihood

# Initialize Hartmann function (4D) and bounds
hartmann = Hartmann(dim=4)
bounds = hartmann.bounds  # [[0, 0, 0, 0], [1, 1, 1, 1]]

# Parameters
n_initial_points = 10
num_queries = 10
num_runs = 10

# Store minimum f(x) for each run
all_min_f_values = []

for run in range(num_runs):
    print(f"\nRun {run + 1}/{num_runs}")
    
    # Set different random seed for each run
    torch.manual_seed(2 + run)
    
    # Initialize with 10 random points in [0, 1]^4
    train_x = torch.rand(size=(n_initial_points, 4))  # Uniformly sampled in [0, 1]^4
    train_y = hartmann(train_x)

    # Print initial points and their function values
    print("Initial points (x1, x2, x3, x4, f(x)):")
    print(torch.hstack([train_x, train_y.unsqueeze(1)]))

    # Fit initial GP model
    model, likelihood = fit_gp_model(train_x, train_y)

    # Track minimum f(x) for this run
    min_f_values = [train_y.min().item()]

    # Run Bayesian optimization
    for i in range(num_queries):
        print(f"Iteration {i}, incumbent x=({train_x[train_y.argmin()].numpy().round(4).tolist()}), f(x)={train_y.min().item():.4f}")

        model, likelihood = fit_gp_model(train_x, train_y)

        policy = botorch.acquisition.analytic.ProbabilityOfImprovement(
            model, best_f=train_y.min(), maximize=False
        )
        
        next_x, acq_val = botorch.optim.optimize_acqf(
            policy,
            bounds=bounds,
            q=1,
            num_restarts=20,
            raw_samples=50,
        )

        next_y = hartmann(next_x)

        # Ensure next_y is 1D with shape (1,)
        next_y = next_y.unsqueeze(0) if next_y.dim() == 0 else next_y

        train_x = torch.cat([train_x, next_x])
        train_y = torch.cat([train_y, next_y])
        
        # Store the minimum f(x)
        min_f_values.append(train_y.min().item())

    # Store results for this run
    all_min_f_values.append(min_f_values)

    # Plot minimum f(x) vs iteration for this run
    plt.figure(figsize=(8, 6))
    plt.plot(range(num_queries + 1), min_f_values, marker='o', linestyle='-', color='b')
    plt.xlabel('Iteration')
    plt.ylabel('Minimum f(x)')
    plt.title(f'Minimum f(x) vs Iteration (Run {run + 1})')
    plt.grid(True)
    plt.savefig(f'min_f_vs_iteration_run_{run + 1}.png')
    plt.close()

# Convert to numpy array for easier computation
all_min_f_values = np.array(all_min_f_values)  # Shape: (num_runs, num_queries + 1)

# Compute mean, std, and 95% CI
mean_f = np.mean(all_min_f_values, axis=0)
std_f = np.std(all_min_f_values, axis=0)
ci_95 = 1.96 * std_f / np.sqrt(num_runs)  # 95% CI: z=1.96 for normal distribution

# Plot convergence with mean, std, and CI
plt.figure(figsize=(10, 6))
iterations = range(num_queries + 1)

# Plot mean
plt.plot(iterations, mean_f, marker='o', linestyle='-', color='b', label='Mean Minimum f(x)')

# Plot 95% CI
plt.fill_between(
    iterations,
    mean_f - ci_95,
    mean_f + ci_95,
    color='b',
    alpha=0.2,
    label='95% Confidence Interval'
)

# Plot mean ± std
plt.fill_between(
    iterations,
    mean_f - std_f,
    mean_f + std_f,
    color='g',
    alpha=0.1,
    label='Mean ± Std'
)

plt.xlabel('Iteration')
plt.ylabel('Minimum f(x)')
plt.title('Convergence Plot: Mean, Std, and 95% CI over 10 Runs (Hartmann 4D)')
plt.grid(True)
plt.legend()
plt.savefig('convergence_plot.png')
plt.close()


Run 1/10
Initial points (x1, x2, x3, x4, f(x)):
tensor([[ 0.6147,  0.3810,  0.6371,  0.4745, -1.8271],
        [ 0.7136,  0.6190,  0.4425,  0.0958, -0.5780],
        [ 0.6142,  0.0573,  0.5657,  0.5332, -1.1070],
        [ 0.3901,  0.9088,  0.5334,  0.7073, -2.7414],
        [ 0.7116,  0.2050,  0.3078,  0.9809, -0.0230],
        [ 0.0103,  0.4660,  0.4604,  0.8547, -0.2548],
        [ 0.4525,  0.6317,  0.4760,  0.2200, -1.8143],
        [ 0.2166,  0.2571,  0.0458,  0.1755, -2.1780],
        [ 0.6177,  0.8291,  0.5246,  0.2708, -0.9913],
        [ 0.7197,  0.3081,  0.3892,  0.2259, -1.3620]])


Training GP:   0%|          | 0/500 [00:00<?, ?it/s]

Iteration 0, incumbent x=([0.39010000228881836, 0.9088000059127808, 0.5333999991416931, 0.7073000073432922]), f(x)=-2.7414


Training GP:   0%|          | 0/500 [00:00<?, ?it/s]

Iteration 1, incumbent x=([0.3889999985694885, 0.9140999913215637, 0.539900004863739, 0.7045000195503235]), f(x)=-2.7538


...

Training GP:   0%|          | 0/500 [00:00<?, ?it/s]

Iteration 8, incumbent x=([0.296999990940094, 0.20360000431537628, 0.6905999779701233, 0.37560001015663147]), f(x)=-3.1840


Training GP:   0%|          | 0/500 [00:00<?, ?it/s]

Iteration 9, incumbent x=([0.2759000062942505, 0.18029999732971191, 0.6852999925613403, 0.3700000047683716]), f(x)=-3.2106


Training GP:   0%|          | 0/500 [00:00<?, ?it/s]